# Pilot Ray Example

In [1]:
# Pilot-Ray
import os, sys, socket
sys.path.insert(0, "../../..")
import pilot.streaming
import ray


#configure loggin
import logging
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger("stevedore.extension").setLevel(logging.CRITICAL)
logging.getLogger("keystoneauth").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

sys.modules['pilot.streaming']

RESOURCE_URL_HPC="slurm+ssh://login03.chn"
WORKING_DIRECTORY=os.path.join(os.environ["HOME"], "work")


## Setup Cluster

In [2]:
# pilot_compute_description = {
#     "resource":RESOURCE_URL_HPC,
#     "working_directory": os.path.join(WORKING_DIRECTORY, "work"),
#     "number_of_nodes": 1, # used for SLURM allocation
#     "cores_per_node": 2, # used for Ray configuration
#     "gpus_per_node": 0,  # used for Ray configuration
#     "scheduler_script_commands": ["#SBATCH --constraint=cpu",
#                                   "#SBATCH --qos regular",
#                                   ],    
#     "walltime": 59,
#     "type":"ray",  
# }

pilot_compute_description = {
    "resource": RESOURCE_URL_HPC,
    "working_directory": WORKING_DIRECTORY,
    "number_cores": 1,
    "queue": "normal",
    "walltime": 5,
    "type": "ray",
    "project": "m4408",
    "os_ssh_keyfile": "~/.ssh/nersc",
    "scheduler_script_commands": ["#SBATCH --constraint=cpu"]
}

In [3]:
ray_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
ray_pilot.wait()

11/25/2023 10:10:15 AM - root - DEBUG - Run python Args: ['-m', 'pilot.plugins.ray.bootstrap_ray', '-j', 'ray-e26da67a-8bbd-11ee-832e-dbc3e3c58676', ' -w ', '/global/homes/l/luckow/work']
11/25/2023 10:10:15 AM - root - DEBUG - Command: python -m pilot.plugins.ray.bootstrap_ray -j ray-e26da67a-8bbd-11ee-832e-dbc3e3c58676  -w  /global/homes/l/luckow/work
11/25/2023 10:10:15 AM - pilot-streaming - DEBUG - Pilot-Job SLURM: Parsing job description: {'executable': 'python', 'arguments': ['-m', 'pilot.plugins.ray.bootstrap_ray', '-j', 'ray-e26da67a-8bbd-11ee-832e-dbc3e3c58676', ' -w ', '/global/homes/l/luckow/work'], 'working_directory': '/global/homes/l/luckow/work', 'output': 'ray_job_ray-e26da67a-8bbd-11ee-832e-dbc3e3c58676.stdout', 'error': 'ray_job_ray-e26da67a-8bbd-11ee-832e-dbc3e3c58676.stderr', 'number_of_nodes': 1, 'cores_per_node': 1, 'project': 'm4408', 'reservation': None, 'queue': 'normal', 'walltime': 5, 'pilot_compute_description': {'resource': 'slurm+ssh://login03.chn', 'work

Working Directory: /global/homes/l/luckow/work
/tmp/tmpegnpr0z4


***************************************************************************
                          NOTICE TO USERS

Lawrence Berkeley National Laboratory operates this computer system under 
contract to the U.S. Department of Energy.  This computer system is the 
property of the United States Government and is for authorized use only.
Users (authorized or unauthorized) have no explicit or implicit 
expectation of privacy.

Any or all uses of this system and all files on this system may be
intercepted, monitored, recorded, copied, audited, inspected, and disclosed
to authorized site, Department of Energy, and law enforcement personnel,
as well as authorized officials of other agencies, both domestic and foreign.
By using this system, the user consents to such interception, monitoring,
recording, copying, auditing, inspection, and disclosure at the discretion
of authorized site or Department of Energy personnel.

Unauthorized or improper use of this system may result in administrative

Submission of Job Command: ssh login03.chn sbatch  tmpegnpr0z4
Cleanup: ssh login03.chn rm tmpegnpr0z4


***************************************************************************
                          NOTICE TO USERS

Lawrence Berkeley National Laboratory operates this computer system under 
contract to the U.S. Department of Energy.  This computer system is the 
property of the United States Government and is for authorized use only.
Users (authorized or unauthorized) have no explicit or implicit 
expectation of privacy.

Any or all uses of this system and all files on this system may be
intercepted, monitored, recorded, copied, audited, inspected, and disclosed
to authorized site, Department of Energy, and law enforcement personnel,
as well as authorized officials of other agencies, both domestic and foreign.
By using this system, the user consents to such interception, monitoring,
recording, copying, auditing, inspection, and disclosure at the discretion
of authorized site or Department of Energy personnel.

Unauthorized or improper use of this system may result in administrative

**** Job: 18649791 State: Queue


11/25/2023 10:10:26 AM - root - DEBUG - Query State: squeue -j 18649791 Output:              JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          18649791 regular_m ps-e26e3   luckow PD       0:00      1 (Resources)

11/25/2023 10:10:26 AM - root - DEBUG - **** Job: 18649791 State: Queue
11/25/2023 10:10:33 AM - root - DEBUG - Query State: squeue -j 18649791 Output:              JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          18649791 regular_m ps-e26e3   luckow PD       0:00      1 (Resources)

11/25/2023 10:10:33 AM - root - DEBUG - **** Job: 18649791 State: Queue
11/25/2023 10:10:39 AM - root - DEBUG - Query State: squeue -j 18649791 Output:              JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          18649791 regular_m ps-e26e3   luckow  R       0:00      1 nid005662

11/25/2023 10:10:39 AM - root - DEBUG - **** Job: 18649791 State: Running
11/25/2023 10:10:39 AM - root - DEBUG - Lookin

init Ray client
Connect to 128.55.76.133:10001


## Ray Test

In [4]:
ray_pilot.get_details()

11/25/2023 10:11:26 AM - root - DEBUG - Results of scheduler startup file check: True


{'master_url': '128.55.76.133:10001',
 'web_ui_url': 'http://128.55.76.133:8265'}

In [5]:
ray_client = ray_pilot.get_context()

11/25/2023 10:11:26 AM - root - DEBUG - Results of scheduler startup file check: True


Connect to 128.55.76.133:10001


In [7]:

# create a remote function
@ray.remote
def f(x):
    return x * x

with ray_client:
    # call the remote function ten times in parallel
    # and return the results as a list
    print(ray.get([f.remote(i) for i in range(10)]))

# Shutdown Cluster

In [ ]:
ray_pilot.cancel()